In [1]:
import os
import glob
import pandas as pd

In [2]:
id_dir = "/home/galaxy/project/m6AQTL/data/data_for_website/final_data/ID_index/ID_table"
map_dir = "/home/galaxy/project/m6AQTL/data/data_for_website/final_data/ID_index/bigtable"
result_dir = "/home/galaxy/project/m6AQTL/data/data_for_website/final_data/ID_index/Mapping_table/RNA_modification"
id_list = glob.glob("%s/*" % id_dir)
map_list = glob.glob("%s/*.txt" % map_dir)
print(len(id_list))
print(map_list)

18
['/home/galaxy/project/m6AQTL/data/data_for_website/final_data/ID_index/bigtable/m5C.txt', '/home/galaxy/project/m6AQTL/data/data_for_website/final_data/ID_index/bigtable/m7G.txt', '/home/galaxy/project/m6AQTL/data/data_for_website/final_data/ID_index/bigtable/Nm.txt', '/home/galaxy/project/m6AQTL/data/data_for_website/final_data/ID_index/bigtable/m6A.txt', '/home/galaxy/project/m6AQTL/data/data_for_website/final_data/ID_index/bigtable/m6Am.txt', '/home/galaxy/project/m6AQTL/data/data_for_website/final_data/ID_index/bigtable/ClinVar.txt', '/home/galaxy/project/m6AQTL/data/data_for_website/final_data/ID_index/bigtable/RNA_Editing.txt', '/home/galaxy/project/m6AQTL/data/data_for_website/final_data/ID_index/bigtable/m1A.txt', '/home/galaxy/project/m6AQTL/data/data_for_website/final_data/ID_index/bigtable/pQTL.txt', '/home/galaxy/project/m6AQTL/data/data_for_website/final_data/ID_index/bigtable/meQTL.txt', '/home/galaxy/project/m6AQTL/data/data_for_website/final_data/ID_index/bigtable/C

In [3]:
id_dict = {}
for id_file in id_list:
    df = pd.read_table(id_file, sep="\t", na_filter = False)
    id_dict[os.path.basename(id_file).split(".")[0]] = dict(zip(df.iloc[:, 0], df.iloc[:, 1]))

In [4]:
def replace_content(in_list, map_dict):
    reversed_map = {v: k for k, v in map_dict.items()}
    result_list = []
    for line in in_list:
        new_line = "!!!!!!!!!!!!!!!!!!!!!!!"
        if line == "NA":
            new_line = "NA"
        else:
            term_list, new_list = line.split("@"), []
            for term in term_list:
                if term == "NA":
                    new_list.append("NA")
                elif term in reversed_map:
                    new_list.append(reversed_map[term])
                else:
                    print(term)
            new_line = "@".join(new_list)
        result_list.append(new_line)
    return result_list

In [5]:
def replace_content_withoutsplit(in_list, map_dict):
    reversed_map = {v: k for k, v in map_dict.items()}
    result_list = []
    for line in in_list:
        new_line = "!!!!!!!!!!!!!!!!!!!!!!!"
        if line == "NA":
            new_line = "NA"
        elif line in reversed_map:
            new_line = reversed_map[line]
        else:
            print(line)
        result_list.append(new_line)
    return result_list

In [6]:
def split_first_column(df):
    # split @joint ID to single ID
    final_columns, final_array = df.columns, []
    for i_index, values in df.iterrows():
        term_id, others = values[0], values[1:].tolist()
        for x in term_id.split("@"):
            new_list = [x] + others
            final_array.append(new_list)
    df_result = pd.DataFrame(final_array, columns=final_columns)
    return df_result

In [7]:
modification_list = ["m1A", "m5C", "m6A", "m7G", "m6Am", "RNA Editing", "pseudoU", "Nm"]
disease_list = ["GWAS", "COSMIC", "ClinVar"]
columns = pd.read_table(os.path.join(map_dir, "eQTL.txt"), sep="\t", na_filter=False).columns
for map_file in map_list:
    prefix = os.path.basename(map_file).split(".")[0].replace("_", " ")
#     if prefix not in disease_list:
    print(prefix)
    df_map = pd.read_table(map_file, sep="\t", na_filter=False)
    first_col, remaining_cols, first_values = df_map.columns[0], list(df_map.columns[1:]), df_map.iloc[:, 0]
    new_contents = []
    if prefix in modification_list:
        new_contents = replace_content_withoutsplit(first_values, id_dict[prefix])
    else:
        new_contents = replace_content(first_values, id_dict[prefix])
    df_map[first_col] = new_contents
    for col in remaining_cols:
        new_contents, feature = [], col.split("_")[0]
        if feature == prefix:
            new_contents = ["NA" for i in range(len(df_map))]
        elif feature in modification_list:
            new_contents = replace_content_withoutsplit(df_map[col], id_dict[feature])
        else:
            new_contents = replace_content(df_map[col], id_dict[feature])
        df_map[col] = new_contents
    result_file = os.path.join(result_dir, os.path.basename(map_file))
    if prefix not in modification_list:
        df_map = split_first_column(df_map)
    df_map.to_csv(result_file, sep="\t", index=False)

m5C
m7G
Nm
m6A
m6Am
ClinVar
RNA Editing
m1A
pQTL
meQTL
COSMIC
pseudoU
GWAS
circQTL
eQTL
sQTL
